In [ ]:
%tensorflow_version 1.x

import tensorflow as tf

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%cd /gdrive/'My Drive'/Projects/Pedestrian_Detection/models/research

/gdrive/My Drive/Projects/Pedestrian_Detection/models/research


In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools
!pip install tf-slim
!pip install numpy==1.17.0

In [ ]:
!pwd
!protoc object_detection/protos/*.proto --python_out=.

/gdrive/My Drive/Projects/Pedestrian_Detection/models/research


In [ ]:
import os
os.environ['PYTHONPATH'] += '/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/:/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/slim/'

!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_tf2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detect

In [ ]:
!mkdir /gdrive/'My Drive'/Projects/Pedestrian_Detection/models/research/pretrained_model
%cd /gdrive/'My Drive'/Projects/Pedestrian_Detection/models/research/pretrained_model

!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xzf ssd_mobilenet_v2_coco_2018_03_29.tar.gz -C .

%cd /gdrive/'My Drive'/Projects/Pedestrian_Detection/models/research/

/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/pretrained_model
--2020-11-08 07:31:34--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.204.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M  43.4MB/s    in 4.4s    

2020-11-08 07:31:40 (40.6 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

/gdrive/My Drive/Projects/Pedestrian_Detection/models/research


In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')
  
# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

import math
import itertools
from itertools import compress
from PIL import Image, ImageDraw

In [ ]:
# Model Name
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = "/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/pretrained_model/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/object_detection/data/mscoco_label_map.pbtxt'

# Load graph
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)


In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.compat.v1.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.compat.v2.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.compat.v2.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.compat.v2.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
def filter_boxes(min_score, boxes, scores, classes, categories):
  """Return boxes with a confidence >= `min_score`"""
  n = len(classes)
  idxs = []
  for i in range(n):
      if classes[i] in categories and scores[i] >= min_score:
          idxs.append(i)
  
  filtered_boxes = boxes[idxs, ...]
  filtered_scores = scores[idxs, ...]
  filtered_classes = classes[idxs, ...]
  return filtered_boxes, filtered_scores, filtered_classes

In [ ]:
def calculate_coord(bbox, width, height):
  """Return boxes coordinates"""
  xmin = bbox[1] * width
  ymin = bbox[0] * height
  xmax = bbox[3] * width
  ymax = bbox[2] * height

  return [xmin, ymin, xmax - xmin, ymax - ymin]
    
def calculate_centr(coord):
  """Calculate centroid for each box"""
  return (coord[0]+(coord[2]/2), coord[1]+(coord[3]/2))
  
def calculate_centr_distances(centroid_1, centroid_2):
  """Calculate the distance between 2 centroids"""
  return  math.sqrt((centroid_2[0]-centroid_1[0])**2 + (centroid_2[1]-centroid_1[1])**2)
  
def calculate_perm(centroids):
  """Return all combinations of centroids"""
  permutations = []
  for current_permutation in itertools.permutations(centroids, 2):
    if current_permutation[::-1] not in permutations:
      permutations.append(current_permutation)
  return permutations
  
def midpoint(p1, p2):
  """Midpoint between 2 points"""
  return ((p1[0] + p2[0])/2, (p1[1] + p2[1])/2)
    
def calculate_slope(x1, y1, x2, y2):
  """Calculate slope"""
  m = (y2-y1)/(x2-x1)
  return m

In [ ]:
import cv2
import matplotlib
from matplotlib import pyplot as plt
plt.ioff()
matplotlib.use('Agg')

FILE_OUTPUT = '/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/test_images/Rail_Station_Predicted2.mp4'

# Playing video from file
cap = cv2.VideoCapture('/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/test_images/Rail_Station_Converted.mp4')

# Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
width = int(cap.get(3))
height = int(cap.get(4))

dim = (width, height)
print(dim)

tope = 10
i = 0
new = True
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        i = 0
        while(cap.isOpened()):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if ret == True:
              # Correct color
              frame = gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(frame, axis=0)

              # Actual detection.
              (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
              
              # Filter boxes
              confidence_cutoff = 0.5
              boxes, scores, classes = filter_boxes(confidence_cutoff, np.squeeze(boxes), np.squeeze(scores), np.squeeze(classes), [1])

              # Calculate normalized coordinates for boxes
              centroids = []
              coordinates = []
              for box in boxes:
                coord = calculate_coord(box, width, height)
                centr = calculate_centr(coord)
                centroids.append(centr)
                coordinates.append(coord)

              # Pixel per meters
              average_px_meter = (width-150) / 7

              permutations = calculate_perm(centroids)

              # Display boxes and centroids
              fig, ax = plt.subplots(figsize = (20,12), dpi = 90, frameon=False)
              ax = fig.add_axes([0, 0, 1, 1])
              ax.axis('off')
              ax.spines['top'].set_visible(False)
              ax.spines['right'].set_visible(False)
              ax.spines['bottom'].set_visible(False)
              ax.spines['left'].set_visible(False)
              ax.get_xaxis().set_ticks([])
              ax.get_yaxis().set_ticks([])
              for coord, centr in zip(coordinates, centroids):
                ax.add_patch(patches.Rectangle((coord[0], coord[1]), coord[2], coord[3], linewidth=2, edgecolor='y', facecolor='none', zorder=10))
                ax.add_patch(patches.Circle((centr[0], centr[1]), 3, color='yellow', zorder=20))

              # Display lines between centroids
              for perm in permutations:
                dist = calculate_centr_distances(perm[0], perm[1])
                dist_m = dist/average_px_meter
                
                x1 = perm[0][0]
                y1 = perm[0][1]
                x2 = perm[1][0]
                y2 = perm[1][1]

                # Calculate middle point
                middle = midpoint(perm[0], perm[1])

                # Calculate slope
                slope = calculate_slope(x1, y1, x2, y2)
                dy = math.sqrt(3**2/(slope**2+1))
                dx = -slope*dy

                # Set random location
                if randrange(10) % 2== 0:
                  Dx = middle[0] - dx*10
                  Dy = middle[1] - dy*10
                else:
                  Dx = middle[0] + dx*10
                  Dy = middle[1] + dy*10

                if dist_m < 1.5:
                  ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='white', xytext=(Dx, Dy), fontsize=10, arrowprops=dict(arrowstyle='->', lw=1.5, color='yellow'), bbox=dict(facecolor='red', edgecolor='white', boxstyle='round', pad=0.2), zorder=35)
                  ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='yellow', zorder=15)
                elif 1.5 < dist_m < 3.5:
                  ax.annotate("{}m".format(round(dist_m, 2)), xy=middle, color='black', xytext=(Dx, Dy), fontsize=8, arrowprops=dict(arrowstyle='->', lw=1.5, color='skyblue'), bbox=dict(facecolor='y', edgecolor='white', boxstyle='round', pad=0.2), zorder=35)
                  ax.plot((perm[0][0], perm[1][0]), (perm[0][1], perm[1][1]), linewidth=2, color='skyblue', zorder=15)
                else:
                  pass
              
              ax.imshow(frame, interpolation='nearest')
              
              # This allows you to save each frame in a folder
              #fig.savefig("/gdrive/My Drive/Projects/Pedestrian_Detection/models/research/test_images/TEST_{}.png".format(i))
              # i += 1

              # Convert figure to numpy
              fig.canvas.draw()

              img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
              img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))

              img = np.array(fig.canvas.get_renderer()._renderer)
              img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

              if new:
                print("Define out")
                out = cv2.VideoWriter(FILE_OUTPUT, cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (img.shape[1], img.shape[0]))
                new = False

              out.write(img)
            else:
              break

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows()

(0, 0)


NameError: ignored